# DPI 1000
from ContaminationExtractor import Paper_Contamination 
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer("jhgan/ko-sroberta-multitask")
POPPLER_PATH = r"C:\Users\aidan\Downloads\ATC\ATC\Result\Step 4\poppler-23.01.0\Library\bin"

save_fullpath = "keyword_100000.csv"
folder_path = "paper_folder"
historical_data_fullpath = "Historical_data.xlsx"
DB_fullpath = "Model_DB.xlsx"
errorfile_fullpath = "ErrorFileList.txt"



Paper_Contamination(save_fullpath, folder_path, historical_data_fullpath, DB_fullpath, errorfile_fullpath)

In [2]:
# DPI 1000
from ContaminationExtractor import Paper_Contamination 


save_fullpath = "keyword_500000.csv"
folder_path = "paper_folder"
historical_data_fullpath = "Historical_data.xlsx"
DB_fullpath = "Model_DB.xlsx"
errorfile_fullpath = "ErrorFileList.txt"



Paper_Contamination(save_fullpath, folder_path, historical_data_fullpath, DB_fullpath, errorfile_fullpath)



[Kss]: Using CPU. Note: This module is much faster with a GPU.
[Kss]: Using CPU. Note: This module is much faster with a GPU.
[Kss]: Using CPU. Note: This module is much faster with a GPU.
[Kss]: Using CPU. Note: This module is much faster with a GPU.
[Kss]: Using CPU. Note: This module is much faster with a GPU.
[Kss]: Using CPU. Note: This module is much faster with a GPU.
[Kss]: Using CPU. Note: This module is much faster with a GPU.
[Kss]: Using CPU. Note: This module is much faster with a GPU.
[Kss]: Using CPU. Note: This module is much faster with a GPU.
[Kss]: Using CPU. Note: This module is much faster with a GPU.
[Kss]: Using CPU. Note: This module is much faster with a GPU.
[Kss]: Using CPU. Note: This module is much faster with a GPU.
[Kss]: Using CPU. Note: This module is much faster with a GPU.
[Kss]: Using CPU. Note: This module is much faster with a GPU.
[Kss]: Using CPU. Note: This module is much faster with a GPU.
[Kss]: Using CPU. Note: This module is much faster with

{"TotalFiles": "100", "ExecutionTimeInSeconds": "4,364"}


In [3]:
for paper in file_names:
    if paper == '.DS_Store':
        continue
    print(f"🧾 OCR 시작: {paper}")
    paper_name = folder_path + '/' + paper

    try:
        data = paper_ocr(paper_name)
        print(f"✅ OCR 성공: {paper} → 총 페이지 수 {len(data)}")
    except Exception as e:
        print(f"❌ OCR 실패: {paper} → {e}")
        errorfile.append(paper_name)
        continue


NameError: name 'file_names' is not defined

In [1]:
def Paper_Contamination(save_fullpath, folder_path, historical_data_fullpath, DB_fullpath, errorfile_fullpath):
    import pandas as pd
    import numpy as np
    from paper_ocr import paper_ocr
    import kss
    import re
    import os
    import warnings
    import time
    import json
    import os
    import glob
    import pandas as pd
    import difflib
    from tqdm import tqdm
    from difflib import SequenceMatcher
    from concurrent.futures import ThreadPoolExecutor, as_completed
    from llama_cpp import Llama

    # 경고 무시
    warnings.filterwarnings("ignore")

    start_time = time.time()

    # Sentence DB
    historical_data = pd.read_excel(
        historical_data_fullpath, engine="openpyxl")
    queries = historical_data.values.tolist()

    # 격실 및 기기 DB
    DB_room_df = pd.read_excel(DB_fullpath, sheet_name='격실 DB')
    DB_mc_df = pd.read_excel(DB_fullpath, sheet_name='기기 DB')

    # 폴더 내의 파일 이름 가져오기 (첫 두 개의 파일만 처리)
    file_names = []
    for file_name in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, file_name)):
            file_names.append(file_name)
    file_names = file_names[:2]  # 첫 두 개 파일만 처리

    # Mistral 모델 로드
    mistral_model_path = "/home/taco/Documents/projects/jung/model/mis/mistral-7b-v0.1.Q4_K_M.gguf"
    llm = Llama(model_path=mistral_model_path, n_ctx=2048,
                n_threads=8, n_gpu_layers=30, verbose=False)

    result_line_list = []
    errorfile = []

    for paper in file_names:
        if paper == '.DS_Store':
            continue
        paper_name = folder_path + '/' + paper

        # OCR
        try:
            data = paper_ocr(paper_name)
        except:
            errorfile.append(paper_name)
            continue

        # 공백 제거
        newdata = []
        for page in data:
            preprocessed_page = [
                re.sub(' +', ' ', sentence).strip() for sentence in page]
            newdata.append(preprocessed_page)

        # 문장 분리
        sentence_temp = []
        if paper_name[-3:].lower() == 'txt' or paper_name[-3:].lower() == 'pdf':
            for j_idx, j in enumerate(newdata):
                sentences = []
                for item in j:
                    sentences.extend(kss.split_sentences(item))
                for sentence_idx, sentence in enumerate(sentences):
                    sentence_temp.append(
                        {'문장': sentence, '페이지': j_idx + 1, '문장위치': sentence_idx + 1})

        # 10자 이하 문장 제거
        min_length = 10
        sentence_temp = [s for s in sentence_temp if len(s['문장']) > min_length]
        sentence = pd.DataFrame(sentence_temp)
        sentence_list = [item['문장'] for item in sentence_temp]

        # 결과 비교
        result_df_list = []
        similarity_cache = {}

        for query_row in queries:
            query = query_row[0]

            with ThreadPoolExecutor(max_workers=6) as executor:
                futures = []

                for target in sentence_list:
                    key = (query, target)
                    if key in similarity_cache:
                        futures.append(executor.submit(
                            lambda q=query, t=target: (q, t, similarity_cache[key])))
                    else:
                        simple_similarity = SequenceMatcher(
                            None, query, target).ratio()
                        if simple_similarity < 0.2:
                            continue
                        futures.append(executor.submit(
                            lambda q=query, t=target: (
                                q, t,
                                float(llm(f"""다음 두 문장은 얼마나 유사합니까? 0부터 1 사이의 점수로 정수나 소수로만 답해주세요.
문장1: {q}
문장2: {t}
점수:""", max_tokens=10, stop=["\n"])['choices'][0]['text'].strip())
                            )
                        ))
                    for future in as_completed(futures):
                        try:
                            q, t, score = future.result()
                            similarity_cache[(q, t)] = score
                            if score > 0.3:
                                result_df_list.append(pd.DataFrame({
                                    'Query': [q],
                                    '오염문장': [t.strip()],
                                    'score': [score]
                                }))
                        except:
                            continue
                        if len(result_df_list) == 0:
                            continue

        # 결과 DataFrame 생성
        if result_df_list:
            result = pd.concat(result_df_list, ignore_index=True)
            result = result.sort_values(by='score', ascending=False)
            result_fin = result.drop_duplicates(['오염문장'], keep='first')
            result_similarity = result_fin.reset_index(drop=True)

            # 페이지, 문장위치 추가
            result_similarity['페이지'] = None
            result_similarity['문장위치'] = None

            for index, row in result_similarity.iterrows():
                query = row['오염문장']
                match_row = sentence[sentence['문장'] == query]
                if not match_row.empty:
                    result_similarity.at[index, '페이지'] = match_row['페이지'].values[0]
                    result_similarity.at[index,
                                         '문장위치'] = match_row['문장위치'].values[0]

            # 결과 출력
            print(f"Paper: {paper}")
            print(result_similarity[['Query', '오염문장', 'score']].head())  # 첫 5개 결과 출력

    # 결과를 CSV로 저장
    result_line = pd.DataFrame(result_line_list)
    try:
        result_line = result_line.sort_values(by='score', ascending=False)
    except:
        pass
    result_line = result_line.drop_duplicates(['오염문장', '장소표현'], keep='first')
    try:
        result_line = result_line.drop(columns=['score'])
    except:
        pass
    result_line = result_line.reset_index(drop=True)

    if result_line.empty:
        result_line['파일이름'] = None
        result_line['오염문장'] = None
        result_line['페이지'] = None
        result_line['문장위치'] = None
        result_line['장소명'] = None
        result_line['기기명'] = None
        result_line['장소표현'] = None
        result_line['기기표현'] = None

    result_line.to_csv(save_fullpath, index=False, encoding='utf-8-sig')

    # 오류 파일 기록
    with open(errorfile_fullpath, 'w') as file:
        for item in errorfile:
            file.write(str(item) + '\n')

    # 실행 종료 시간 기록
    end_time = time.time()

    # 실행 시간 계산
    execution_time = end_time - start_time
    total_files = len(file_names)

    # JSON 형식으로 출력
    JSON_text = {
        "TotalFiles": f"{total_files:,}",
        "ExecutionTimeInSeconds": f"{execution_time:,.0f}"
    }

    json_data = json.dumps(JSON_text)
    print(json_data)


In [4]:
from pdf2image import convert_from_path
from PIL import Image
import numpy as np
import easyocr
import traceback
import os
os.environ["LD_LIBRARY_PATH"] = "/home/taco/Documents/projects/jung/poppler_2301_build/lib"

def paper_ocr(paper_name):
    try:
        # GPU 사용 설정, 모델 경로 명시
        reader = easyocr.Reader(
            ['en', 'ko'],
            gpu=False,
            model_storage_directory='model',
            user_network_directory='model',
            download_enabled=False
        )

        # .txt 파일 처리
        if paper_name[-3:].lower() == 'txt':
            with open(paper_name, 'r', encoding='utf-8') as f:
                paper_temp = f.read()
                paper_temp = [[paper_temp]]

        # .pdf 파일 처리
        elif paper_name[-3:].lower() == 'pdf':
            pages = convert_from_path(
                paper_name,
                dpi=800,
                poppler_path="/home/taco/Documents/projects/jung/poppler_2301_build/bin"  # 리눅스 poppler 경로 (환경에 따라 조정 가능)
            )
            paper_temp = []
            for page in pages:
                image_array = np.array(page)
                result = reader.readtext(image_array, detail=0, paragraph=1)
                paper_temp.append(result)

        return paper_temp

    except Exception as e:
        print("❌ 변환 실패!")
        traceback.print_exc()
        return None

# 테스트 예시 (경로 수정해서 사용하세요)
output = paper_ocr("/home/taco/Documents/projects/jung/model/paper_folder/[고리4호기]증기발생기 'A' 수실 배수관의 배수밸브 용접부 누설부위 정비를 위한 원자로 수동정지.pdf")

# 결과 확인
if output is not None:
    for i, page in enumerate(output):
        print(f"\n--- Page {i+1} ---")
        print("\n".join(page))


[Kss]: Using CPU. Note: This module is much faster with a GPU.


KeyboardInterrupt: 

In [5]:
import os
import pandas as pd
import numpy as np
import re
import kss
import traceback
from llama_cpp import Llama
from difflib import SequenceMatcher
from paper_ocr import paper_ocr  # OCR 함수 사용 가능 상태여야 함

# 🔧 설정
folder_path = "paper_folder"
historical_data_fullpath = "Historical_data.xlsx"
DB_fullpath = "Model_DB.xlsx"
mistral_model_path = "/home/taco/Documents/projects/jung/model/mis/mistral-7b-v0.1.Q4_K_M.gguf"

# 점수 추출 함수
def extract_score(text):
    try:
        match = re.search(r"([0-9]+\.?[0-9]*)", text)
        if match:
            return float(match.group(1))
    except:
        return None
    return None

# Step 0. 첫 파일만 선택
file_names = [f for f in os.listdir(folder_path) if f.endswith('.pdf') and not f.startswith('.')]
if not file_names:
    raise FileNotFoundError("❌ PDF 파일이 없습니다.")

paper = file_names[0]
paper_path = os.path.join(folder_path, paper)
print(f"\n📄 선택된 파일: {paper_path}")

# Step 1. LLM 모델 로드
print("🔄 LLM 모델 로드 중...")
llm = Llama(model_path=mistral_model_path, n_ctx=512, n_threads=4, n_gpu_layers=0, verbose=True)
print("✅ LLM 모델 로드 완료\n")

# Step 2. Sentence DB
print("📥 Sentence DB 로딩 중...")
queries = pd.read_excel(historical_data_fullpath, engine="openpyxl").values.tolist()
print(f"✅ 문장 쿼리 수: {len(queries)}")

# Step 3. OCR
try:
    print("🔍 OCR 시작...")
    data = paper_ocr(paper_path)
    print("✅ OCR 성공")
except Exception as e:
    print("❌ OCR 실패")
    traceback.print_exc()
    raise e

# Step 4. 공백 제거
newdata = []
for page in data:
    preprocessed = [re.sub(' +', ' ', s).strip() for s in page]
    newdata.append(preprocessed)
print(f"✅ OCR 후 페이지 수: {len(newdata)}")

# Step 5. 문장 분리
sentence_temp = []
for j_idx, page in enumerate(newdata):
    sentences = []
    for item in page:
        sentences.extend(kss.split_sentences(item))
    for sentence_idx, sentence in enumerate(sentences):
        sentence_temp.append({'문장': sentence, '페이지': j_idx+1, '문장위치': sentence_idx+1})
print(f"✅ 분리된 문장 수: {len(sentence_temp)}")

# Step 6. 10자 이상 필터
sentence_temp = [s for s in sentence_temp if len(s['문장']) > 10]
sentence_list = [s['문장'] for s in sentence_temp]
print(f"✅ 10자 이상 문장 수: {len(sentence_list)}")

# Step 7. LLM 유사도 비교 (1개만)
print("\n🔬 LLM 유사도 테스트")
query = queries[0][0].strip()
target = sentence_list[0].strip()

print(f"🔸 Query 문장:\n{query}")
print(f"🔹 Target 문장:\n{target}")

sim_score = SequenceMatcher(None, query, target).ratio()
print(f"🔁 SequenceMatcher 유사도: {sim_score:.3f}")
if sim_score < 0.3:
    print("⛔ 유사도 낮아서 스킵")
else:
    prompt = f"""두 문장의 유사도를 0에서 1 사이 소수점 점수로만 숫자 하나만 출력하세요. 다른 말은 하지 마세요.\n문장1: {query}\n문장2: {target}\n답:"""
    try:
        output = llm(prompt, max_tokens=10, stop=["\n"])
        score = extract_score(output['choices'][0]['text'])
        print(f"✅ LLM 유사도 점수: {score}")
    except:
        print("❌ LLM 호출 실패")
        traceback.print_exc()


llama_model_load_from_file_impl: using device CUDA0 (NVIDIA GeForce RTX 3080 Laptop GPU) - 7619 MiB free
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /home/taco/Documents/projects/jung/model/mis/mistral-7b-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: 


📄 선택된 파일: paper_folder/주증기모관 배기밸브 정비, 저압터빈 날개 점검, MSR Expansion Bellows Joint 점검 등을 위한 원자로 수....pdf
🔄 LLM 모델 로드 중...
✅ LLM 모델 로드 완료

📥 Sentence DB 로딩 중...
✅ 문장 쿼리 수: 136
🔍 OCR 시작...
✅ OCR 성공
✅ OCR 후 페이지 수: 1
✅ 분리된 문장 수: 19
✅ 10자 이상 문장 수: 5

🔬 LLM 유사도 테스트
🔸 Query 문장:
원자로건물 2층의 냉각수 저장탱크에서 누설이 발생하여 방사능 감지기가 작동하였다. 작업자 B가 탱크의 밸브를 잘못 조작한 것이 원인으로 파악되었다.
🔹 Target 문장:
사건발생일자 85/10/19 23:58
🔁 SequenceMatcher 유사도: 0.058
⛔ 유사도 낮아서 스킵


In [ ]:
from pdf2image import convert_from_path
from PIL import Image
import numpy as np
import easyocr
import difflib

def compare_ocr_outputs(pdf_path, poppler_path="/usr/bin"):
    # PDF → 이미지 변환
    pages = convert_from_path(pdf_path, dpi=800, poppler_path=poppler_path)

    results = {"cpu": [], "gpu": []}

    for gpu_mode in [False, True]:
        reader = easyocr.Reader(['en', 'ko'], gpu=gpu_mode,
                                model_storage_directory='model',
                                user_network_directory='model',
                                download_enabled=False)
        key = "gpu" if gpu_mode else "cpu"

        for page in pages:
            image_array = np.array(page)
            text = reader.readtext(image_array, detail=0, paragraph=1)
            results[key].append("\n".join(text))

    return results

def show_differences(cpu_output, gpu_output):
    print("=== 🔍 Difference between CPU and GPU OCR ===\n")
    for page_idx, (cpu_text, gpu_text) in enumerate(zip(cpu_output, gpu_output)):
        print(f"\n--- Page {page_idx + 1} ---")
        cpu_lines = cpu_text.splitlines()
        gpu_lines = gpu_text.splitlines()
        diff = difflib.unified_diff(cpu_lines, gpu_lines, fromfile='CPU', tofile='GPU', lineterm='')
        for line in diff:
            print(line)

# 사용 예시
pdf_path = "/home/taco/Documents/projects/jung/model/paper_folder/[고리4호기]증기발생기 'A' 수실 배수관의 배수밸브 용접부 누설부위 정비를 위한 원자로 수동정지.pdf"

result = compare_ocr_outputs(pdf_path)
show_differences(result["cpu"], result["gpu"])


In [ ]:
import pandas as pd
import torch
import os
from ContaminationExtractor import Paper_Contamination  # 외부 모듈 불러오기

def Paper_Contamination_Compare(gpu_result_path, cpu_result_path):
    print("\n📊 [결과 비교 중]")

    gpu_df = pd.read_csv(gpu_result_path)
    cpu_df = pd.read_csv(cpu_result_path)

    gpu_sentences = set(gpu_df['오염문장'].dropna().str.strip().unique())
    cpu_sentences = set(cpu_df['오염문장'].dropna().str.strip().unique())

    print(f"총 문장 수 (GPU): {len(gpu_sentences)}")
    print(f"총 문장 수 (CPU): {len(cpu_sentences)}")

    gpu_only = gpu_sentences - cpu_sentences
    cpu_only = cpu_sentences - gpu_sentences

    print(f"\n🔍 GPU/CPU 결과에서 서로 다른 오염문장 수: {len(gpu_only) + len(cpu_only)}")
    if not gpu_only and not cpu_only:
        print("✅ GPU와 CPU 결과는 **완전히 동일합니다.**")
    else:
        print("❗ 차이 발생!")
        if gpu_only:
            print("\n🔺 GPU에만 있는 문장 (예시 최대 5개):")
            for sent in list(gpu_only)[:5]:
                print(" -", sent)
        if cpu_only:
            print("\n🔻 CPU에만 있는 문장 (예시 최대 5개):")
            for sent in list(cpu_only)[:5]:
                print(" -", sent)


def run_contamination_dual():
    # 경로 설정 (사용자 입력 반영)
    folder_path = "paper_folder"
    historical_data_fullpath = "Historical_data.xlsx"
    DB_fullpath = "Model_DB.xlsx"

    # GPU 경로
    gpu_result = "keyword_100000_gpu.csv"
    gpu_error = "ErrorFileList_gpu.txt"

    # CPU 경로
    cpu_result = "keyword_100000_cpu.csv"
    cpu_error = "ErrorFileList_cpu.txt"

    # 1. GPU 실행
    print("▶ GPU 실행 중...")
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # GPU 지정
    Paper_Contamination(
        save_fullpath=gpu_result,
        folder_path=folder_path,
        historical_data_fullpath=historical_data_fullpath,
        DB_fullpath=DB_fullpath,
        errorfile_fullpath=gpu_error
    )

    # 2. CPU 실행
    print("\n▶ CPU 실행 중...")
    os.environ["CUDA_VISIBLE_DEVICES"] = ""  # CPU 강제 지정
    Paper_Contamination(
        save_fullpath=cpu_result,
        folder_path=folder_path,
        historical_data_fullpath=historical_data_fullpath,
        DB_fullpath=DB_fullpath,
        errorfile_fullpath=cpu_error
    )

    # 3. 결과 비교
    Paper_Contamination_Compare(gpu_result, cpu_result)

# 실행
run_contamination_dual()


In [ ]:
from pdf2image import convert_from_path
from PIL import Image
import numpy as np
import easyocr
import torch

def compare_ocr_outputs(pdf_path, poppler_path="/usr/bin"):
    results = {"cpu": [], "gpu": []}

    # PDF -> 이미지
    pages = convert_from_path(pdf_path, dpi=400, poppler_path=poppler_path)

    # 리더 생성
    reader_cpu = easyocr.Reader(['en', 'ko'], gpu=False, model_storage_directory='model', user_network_directory='model', download_enabled=False)
    reader_gpu = easyocr.Reader(['en', 'ko'], gpu=True, model_storage_directory='model', user_network_directory='model', download_enabled=False)

    for i, page in enumerate(pages):
        image_array = np.array(page)

        # CPU OCR
        try:
            text_cpu = reader_cpu.readtext(image_array, detail=0, paragraph=1)
            results["cpu"].append("\n".join(text_cpu))
        except Exception as e:
            results["cpu"].append(f"[CPU OCR 실패: {str(e)}]")

        # GPU OCR
        try:
            text_gpu = reader_gpu.readtext(image_array, detail=0, paragraph=1)
            results["gpu"].append("\n".join(text_gpu))
        except RuntimeError as e:
            if "CUDA out of memory" in str(e):
                results["gpu"].append("[GPU OCR 실패 - OOM]")
            else:
                results["gpu"].append(f"[GPU OCR 실패: {str(e)}]")
        finally:
            torch.cuda.empty_cache()  # GPU 캐시 비우기

    return results

def show_differences(cpu_result, gpu_result):
    for i, (cpu, gpu) in enumerate(zip(cpu_result, gpu_result)):
        print(f"\n📝 Page {i+1}")
        print("🔵 CPU:")
        print(cpu)
        print("🟢 GPU:")
        print(gpu)

# 🔽 여기에 PDF 경로만 넣으면 실행됨
pdf_path = "/home/taco/Documents/projects/jung/model/paper_folder/[고리4호기]증기발생기 'A' 수실 배수관의 배수밸브 용접부 누설부위 정비를 위한 원자로 수동정지.pdf"
result = compare_ocr_outputs(pdf_path)
show_differences(result["cpu"], result["gpu"])


In [ ]:
import pandas as pd
import torch
import os
from contamination import Paper_Contamination  # 기존 함수 불러온다고 가정

def Paper_Contamination_Compare(gpu_result_path, cpu_result_path):
    print("\n📊 [결과 비교 중]")

    gpu_df = pd.read_csv(gpu_result_path)
    cpu_df = pd.read_csv(cpu_result_path)

    gpu_sentences = set(gpu_df['오염문장'].dropna().str.strip().unique())
    cpu_sentences = set(cpu_df['오염문장'].dropna().str.strip().unique())

    print(f"총 문장 수 (GPU): {len(gpu_sentences)}")
    print(f"총 문장 수 (CPU): {len(cpu_sentences)}")

    gpu_only = gpu_sentences - cpu_sentences
    cpu_only = cpu_sentences - gpu_sentences

    print(f"\n🔍 GPU/CPU 결과에서 서로 다른 오염문장 수: {len(gpu_only) + len(cpu_only)}")
    if not gpu_only and not cpu_only:
        print("✅ GPU와 CPU 결과는 **완전히 동일합니다.**")
    else:
        print("❗ 차이 발생!")
        if gpu_only:
            print("\n🔺 GPU에만 있는 문장 (예시 최대 5개):")
            for sent in list(gpu_only)[:5]:
                print(" -", sent)
        if cpu_only:
            print("\n🔻 CPU에만 있는 문장 (예시 최대 5개):")
            for sent in list(cpu_only)[:5]:
                print(" -", sent)


def run_contamination_dual():
    # 공통 설정
    paper_folder = "/home/taco/Documents/projects/jung/model/paper_folder"
    historical_data = "Historical_data.xlsx"
    DB_path = "Model_DB.xlsx"
    result_dir = "/home/taco/Documents/projects/jung/model/results"
    os.makedirs(result_dir, exist_ok=True)

    # GPU 실행
    print("▶ GPU 실행 중...")
    gpu_result = os.path.join(result_dir, "gpu_result.csv")
    gpu_error = os.path.join(result_dir, "gpu_error.txt")
    torch.cuda.set_device(0) if torch.cuda.is_available() else None
    Paper_Contamination(
        save_fullpath=gpu_result,
        folder_path=paper_folder,
        historical_data_fullpath=historical_data,
        DB_fullpath=DB_path,
        errorfile_fullpath=gpu_error
    )

    # CPU 실행
    print("\n▶ CPU 실행 중...")
    cpu_result = os.path.join(result_dir, "cpu_result.csv")
    cpu_error = os.path.join(result_dir, "cpu_error.txt")
    torch.cuda.set_device(-1)  # CPU 강제 지정 (optional, 실제 모델 내에서 device 고정해야 의미 있음)
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    Paper_Contamination(
        save_fullpath=cpu_result,
        folder_path=paper_folder,
        historical_data_fullpath=historical_data,
        DB_fullpath=DB_path,
        errorfile_fullpath=cpu_error
    )

    # 결과 비교
    Paper_Contamination_Compare(gpu_result, cpu_result)

# 실행
run_contamination_dual()


In [ ]:
from pdf2image import convert_from_path
from PIL import Image
import numpy as np
import easyocr
import torch

def compare_ocr_outputs(pdf_path, poppler_path="/usr/bin"):
    results = {"cpu": [], "gpu": []}

    # PDF -> 이미지
    pages = convert_from_path(pdf_path, dpi=400, poppler_path=poppler_path)

    # 리더 생성
    reader_cpu = easyocr.Reader(['en', 'ko'], gpu=False, model_storage_directory='model', user_network_directory='model', download_enabled=False)
    reader_gpu = easyocr.Reader(['en', 'ko'], gpu=True, model_storage_directory='model', user_network_directory='model', download_enabled=False)

    for i, page in enumerate(pages):
        image_array = np.array(page)

        # CPU OCR
        try:
            text_cpu = reader_cpu.readtext(image_array, detail=0, paragraph=1)
            results["cpu"].append("\n".join(text_cpu))
        except Exception as e:
            results["cpu"].append(f"[CPU OCR 실패: {str(e)}]")

        # GPU OCR
        try:
            text_gpu = reader_gpu.readtext(image_array, detail=0, paragraph=1)
            results["gpu"].append("\n".join(text_gpu))
        except RuntimeError as e:
            if "CUDA out of memory" in str(e):
                results["gpu"].append("[GPU OCR 실패 - OOM]")
            else:
                results["gpu"].append(f"[GPU OCR 실패: {str(e)}]")
        finally:
            torch.cuda.empty_cache()  # GPU 캐시 비우기

    return results

def show_differences(cpu_result, gpu_result):
    for i, (cpu, gpu) in enumerate(zip(cpu_result, gpu_result)):
        print(f"\n📝 Page {i+1}")
        print("🔵 CPU:")
        print(cpu)
        print("🟢 GPU:")
        print(gpu)

# 🔽 여기에 PDF 경로만 넣으면 실행됨
pdf_path = "/home/taco/Documents/projects/jung/model/paper_folder/[고리4호기]증기발생기 'A' 수실 배수관의 배수밸브 용접부 누설부위 정비를 위한 원자로 수동정지.pdf"
result = compare_ocr_outputs(pdf_path)
show_differences(result["cpu"], result["gpu"])


In [ ]:
import os
from pdf2image import convert_from_path
import numpy as np
import easyocr

POPPLER_PATH = "/usr/bin"

# 기존 OCR 함수 그대로 유지
def paper_ocr(paper_name):
    reader = easyocr.Reader(['en', 'ko'], gpu=True, model_storage_directory='model', user_network_directory='model', download_enabled=False)
    
    if paper_name.endswith('.txt'):
        with open(paper_name, 'r', encoding='utf-8') as f:
            return [[f.read()]]  # 기존과 동일하게 리스트 형태로 반환

    elif paper_name.endswith('.pdf'):
        pages = convert_from_path(paper_name, dpi=800, poppler_path=POPPLER_PATH)
        paper_temp = []
        for page in pages:
            image_array = np.array(page)
            result = reader.readtext(image_array, detail=0, paragraph=False)  # 기존 코드와 일치
            paper_temp.append(result)
        return paper_temp

# 기존 OCR 함수를 사용하여 TXT 파일로 변환
def convert_pdf_to_txt(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    file_names = [f for f in os.listdir(input_folder) if f.endswith('.pdf')]

    for pdf_file in file_names:
        pdf_path = os.path.join(input_folder, pdf_file)
        txt_filename = pdf_file.replace('.pdf', '.txt')
        txt_path = os.path.join(output_folder, txt_filename)

        # OCR 실행 (기존 paper_ocr과 동일한 방식)
        try:
            ocr_result = paper_ocr(pdf_path)
            extracted_text = ["\n".join(page) for page in ocr_result]

            # 기존 문장 구조와 동일한 방식으로 저장
            with open(txt_path, 'w', encoding='utf-8') as f:
                for page_num, text in enumerate(extracted_text, start=1):
                    f.write(f"[PAGE {page_num}]\n{text}\n\n")  # 기존 방식 유지

            print(f"✅ {pdf_file} → {txt_filename} 변환 완료")

        except Exception as e:
            print(f"❌ {pdf_file} 변환 실패: {e}")

# 실행
input_folder = "paper_folder"
output_folder = os.path.join(input_folder, "txt_output")
convert_pdf_to_txt(input_folder, output_folder)


In [ ]:
from llama_cpp import Llama

GGUF_MODEL_PATH = r"C:\Users\aidan\Downloads\ATC\ATC\Result\Step 4\model\mis\mistral-7b-v0.1.Q4_K_M.gguf"

try:
    llm = Llama(
        model_path=GGUF_MODEL_PATH, 
        n_ctx=2048, 
        n_gpu_layers=0,  # GPU 미사용
        n_batch=128,  # 배치 크기 줄이기
        n_threads=4   # CPU 스레드 수 제한
    )
    print("✅ GGUF 모델 로드 성공!")
except Exception as e:
    print(f"❌ GGUF 모델 로드 실패: {e}")
import os
print(os.path.exists(GGUF_MODEL_PATH))

In [5]:
import MeCab
tagger = MeCab.Tagger()
print(tagger.parse("형태소 분석이 잘 되나요?"))


형태소	NNG,*,F,형태소,Compound,*,*,형태/NNG/*+소/NNG/*
분석	NNG,행위,T,분석,*,*,*,*
이	JKS,*,F,이,*,*,*,*
잘	MAG,*,T,잘,*,*,*,*
되	VV,*,F,되,*,*,*,*
나요	EF,*,F,나요,*,*,*,*
?	SF,*,*,*,*,*,*,*
EOS



In [ ]:
import os
import pandas as pd
import torch
from pdf2image import convert_from_path
from PIL import Image
import numpy as np
import easyocr
from llama_cpp import Llama  # GGUF 모델 실행을 위한 라이브러리
import difflib

# GGUF 모델 설정 (Mistral-7B GGUF 사용)
GGUF_MODEL_PATH = r"C:\Users\aidan\Downloads\ATC\ATC\Result\Step 4\model\mis\mistral-7b-v0.1.Q2_K_M.gguf"

# GGUF 모델 파일 존재 여부 확인
if not os.path.exists(GGUF_MODEL_PATH):
    raise FileNotFoundError(f"GGUF 모델이 {GGUF_MODEL_PATH}에 존재하지 않습니다. 다운로드 후 다시 실행하세요.")

# GGUF 모델 로드 (CPU 실행)
llm = Llama(model_path=GGUF_MODEL_PATH, n_ctx=2048)

POPPLER_PATH = r"C:\Users\aidan\Downloads\ATC\ATC\Result\Step 4\poppler-23.01.0\Library\bin"

# 결과 저장 경로
save_fullpath = "keyword_llm_gguf.csv"
folder_path = "paper_folder"
DB_fullpath = "Model_DB.xlsx"
errorfile_fullpath = "ErrorFileList_llm_gguf.txt"

def paper_ocr(paper_name):
    """
    PDF 또는 TXT 파일에서 텍스트를 추출하는 OCR 함수.
    """
    reader = easyocr.Reader(['en', 'ko'], gpu=False)

    # .txt 파일 처리
    if paper_name.lower().endswith('.txt'):
        with open(paper_name, 'r', encoding='utf-8') as f:
            paper_temp = [[f.read()]]
        return paper_temp

    # .pdf 파일 처리 (OCR)
    elif paper_name.lower().endswith('.pdf'):
        pages = convert_from_path(paper_name, 800, poppler_path=POPPLER_PATH)
        paper_temp = []
        for page in pages:
            image_array = np.array(page)
            result = reader.readtext(image_array, detail=0, paragraph=1)
            paper_temp.append(result)
        return paper_temp

    return []

def extract_contamination_with_llm(data, DB_room_df, DB_mc_df):
    """
    GGUF 기반 LLM을 이용하여 격실 및 기기명 정보를 추출하고 DB와 매핑하는 함수.
    
    :param data: OCR에서 추출된 텍스트 데이터 (페이지별 리스트)
    :param DB_room_df: 격실 DB
    :param DB_mc_df: 기기 DB
    :return: [{파일이름, 문장, 페이지, 문장위치, 격실, 기기명, 장소표현, 기기표현}...]
    """
    extracted_results = []

    # DB 데이터 로드
    room_num = DB_room_df['격실번호'].astype(str).tolist()
    room_en = DB_room_df['영문명'].astype(str).tolist()
    room_ko = DB_room_df['한글명'].astype(str).tolist()
    mc_num = DB_mc_df['기능위치'].astype(str).tolist()
    mc_name = DB_mc_df['기능위치명'].astype(str).tolist()
    mc_room = DB_mc_df['설치룸'].astype(str).tolist()

    for page_idx, page_content in enumerate(data):
        text_block = " ".join(page_content)

        # GGUF 모델을 이용한 LLM 실행
        try:
            response = llm(text_block, max_tokens=256)
            if "choices" in response and isinstance(response["choices"], list) and len(response["choices"]) > 0:
                extracted_text = response["choices"][0]["text"]
            else:
                extracted_text = ""
        except Exception as e:
            print(f"LLM 처리 오류: {e}")
            continue

        # DB와 매칭 수행
        matched_sentences = []
        for sentence in extracted_text.split("\n"):
            matched_row = {"문장": sentence.strip(), "페이지": page_idx + 1, "문장위치": len(matched_sentences) + 1, "장소명": "", "기기명": "", "장소표현": "", "기기표현": ""}

            # 격실명 매칭
            for word in room_ko + room_en + room_num:
                if word in sentence:
                    matched_row["장소명"] = word
                    matched_row["장소표현"] = word

            # 기기명 매칭
            for word in mc_name + mc_num:
                if word in sentence:
                    matched_row["기기명"] = word
                    matched_row["기기표현"] = word

            matched_sentences.append(matched_row)

        extracted_results.extend(matched_sentences)

    return extracted_results

# 실행 부분
file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# DB 로드
DB_room_df = pd.read_excel(DB_fullpath, sheet_name='격실 DB')
DB_mc_df = pd.read_excel(DB_fullpath, sheet_name='기기 DB')

result_line_list = []
errorfile = []

for paper in file_names:
    paper_name = os.path.join(folder_path, paper)

    try:
        data = paper_ocr(paper_name)
    except Exception as e:
        errorfile.append(paper_name)
        continue

    extracted_results = extract_contamination_with_llm(data, DB_room_df, DB_mc_df)

    for row in extracted_results:
        row["파일이름"] = paper_name
        result_line_list.append(row)

# 결과 저장
result_df = pd.DataFrame(result_line_list)
result_df.to_csv(save_fullpath, index=False, encoding='utf-8-sig')

# 오류 파일 목록 저장
with open(errorfile_fullpath, "w") as file:
    for item in errorfile:
        file.write(str(item) + "\n")

print("GGUF 기반 LLM 격실 및 기기명 추출 완료.")


In [ ]:
import os
import json
import re
import pandas as pd
import numpy as np
import torch
from pdf2image import convert_from_path
from PIL import Image
import easyocr
from llama_cpp import Llama

# GGUF 모델 설정
GGUF_MODEL_PATH = r"C:\Users\aidan\Downloads\ATC\ATC\Result\Step 4\model\mis\mistral-7b-v0.1.Q4_K_M.gguf"

# GGUF 모델 파일 확인
if not os.path.exists(GGUF_MODEL_PATH):
    raise FileNotFoundError(f"GGUF 모델이 {GGUF_MODEL_PATH}에 없습니다. 다운로드 후 다시 실행하세요.")

# GGUF 모델 로드 (CPU 실행)
llm = Llama(model_path=GGUF_MODEL_PATH, n_ctx=2048)

# PDF OCR 관련 설정
POPPLER_PATH = r"C:\Users\aidan\Downloads\ATC\ATC\Result\Step 4\poppler-23.01.0\Library\bin"

# 결과 저장 경로
save_fullpath = "keyword_llm_gguf.csv"
folder_path = "paper_folder"
DB_fullpath = "Model_DB.xlsx"
errorfile_fullpath = "ErrorFileList_llm_gguf.txt"

def paper_ocr(paper_name):
    """PDF 또는 TXT 파일에서 텍스트를 추출하는 OCR 함수"""
    reader = easyocr.Reader(['en', 'ko'], gpu=False)

    if paper_name.lower().endswith('.txt'):
        with open(paper_name, 'r', encoding='utf-8') as f:
            return [[f.read()]]

    elif paper_name.lower().endswith('.pdf'):
        pages = convert_from_path(paper_name, 800, poppler_path=POPPLER_PATH)
        paper_temp = []
        for page in pages:
            image_array = np.array(page)
            result = reader.readtext(image_array, detail=0, paragraph=1)
            paper_temp.append(result)
        return paper_temp

    return []

def extract_json_from_text(text):
    """LLM이 반환한 텍스트에서 JSON 부분만 추출하는 함수"""
    match = re.search(r'\{[\s\S]*?\}', text)  # JSON 패턴 찾기
    if match:
        return match.group()
    return None  # JSON이 없으면 None 반환

def extract_contamination_with_llm(data, DB_room_df, DB_mc_df):
    """
    GGUF 기반 LLM을 이용하여 원자력 관련 격실 및 기기명 정보를 추출하고 DB와 매칭하는 함수.
    """
    extracted_results = []

    # DB에서 장소명과 기기명 리스트 생성 (결측치 제거 후 변환)
    room_list = DB_room_df["장소명"].dropna().astype(str).tolist()
    mc_list = DB_mc_df["기기명"].dropna().astype(str).tolist()

    for page_idx, page_content in enumerate(data):
        for sentence in page_content:  # 문장 단위로 LLM 실행
            sentence = str(sentence).strip()  # 문장이 NaN이거나 float인 경우 대비

            # 1. 문장에서 DB의 장소명/기기명을 찾기
            matched_rooms = [room for room in room_list if room in sentence]
            matched_mcs = [mc for mc in mc_list if mc in sentence]

            # 2. 장소명/기기명이 없으면 LLM 호출하지 않음
            if not matched_rooms and not matched_mcs:
                continue

            # 3. LLM 프롬프트에서 JSON 출력 강제
            prompt = f"""
            🔹 문장에서 장소명과 기기명을 JSON 형식으로 출력하세요.

            **📌 주의사항**
            - **JSON 외의 설명을 절대 포함하지 마세요.**
            - 장소명은 '격실 DB'의 '한글명' 컬럼에서 가져온 값과 일치해야 합니다.
            - 기기명은 '기기 DB'의 '기능위치명' 컬럼에서 가져온 값과 일치해야 합니다.

            **📜 문장 예시**
            "{sentence}"

            **✅ JSON 형식 예제**
            ```json
            {{
                "장소명": "{matched_rooms[0] if matched_rooms else '미확인'}",
                "기기명": "{matched_mcs[0] if matched_mcs else '미확인'}",
                "장소표현": "원문에서 추출된 표현",
                "기기표현": "원문에서 추출된 표현"
            }}
            ```
            """

            try:
                response = llm.create_completion(
                    prompt=prompt,
                    max_tokens=500,  # 충분한 응답 공간 확보
                    stop=["\n\n", "```"]  # 코드 블록 종료 강제
                )
                extracted_text = response["choices"][0]["text"] if "choices" in response else ""

                # JSON 추출 및 검증
                json_text = extract_json_from_text(extracted_text)

                if json_text:
                    try:
                        extracted_info = json.loads(json_text)
                    except json.JSONDecodeError:
                        print(f"⚠️ JSON 변환 오류 발생. 원본 출력: {json_text}")
                        extracted_info = {}
                else:
                    print("⚠️ JSON 형식 감지 실패. LLM 응답 문제 가능성 있음.")
                    extracted_info = {}

                # 결과 정리
                matched_row = {
                    "문장": sentence.strip(),
                    "페이지": page_idx + 1,
                    "문장위치": len(extracted_results) + 1,
                    "장소명": extracted_info.get("장소명", ""),
                    "기기명": extracted_info.get("기기명", ""),
                    "장소표현": extracted_info.get("장소표현", ""),
                    "기기표현": extracted_info.get("기기표현", "")
                }
                extracted_results.append(matched_row)

            except Exception as e:
                print(f"⚠️ LLM 처리 오류: {e}")
                continue

    return extracted_results

# 실행 부분
file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# DB 불러오기
DB_room_df = pd.read_excel(DB_fullpath, sheet_name='격실 DB')
DB_mc_df = pd.read_excel(DB_fullpath, sheet_name='기기 DB')

# 컬럼명 자동 확인 후 매핑
DB_room_df.rename(columns={"한글명": "장소명"}, inplace=True)
DB_mc_df.rename(columns={"기능위치명": "기기명"}, inplace=True)

result_line_list = []
errorfile = []

for paper in file_names:
    paper_name = os.path.join(folder_path, paper)

    try:
        data = paper_ocr(paper_name)
    except Exception as e:
        errorfile.append(paper_name)
        continue

    extracted_results = extract_contamination_with_llm(data, DB_room_df, DB_mc_df)

    for row in extracted_results:
        row["파일이름"] = paper_name
        result_line_list.append(row)

# 결과 저장
result_df = pd.DataFrame(result_line_list)
result_df.to_csv(save_fullpath, index=False, encoding='utf-8-sig')

print("✅ GGUF 기반 LLM 격실 및 기기명 추출 완료.")


In [ ]:
import torch
import torchvision
print(torch.__version__)  # torch 버전 출력
print(torchvision.__version__)  # torchvision 버전 출력


In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("jhgan/ko-sroberta-multitask")
sentences = ["안녕하세요", "이 문장은 테스트를 위한 것입니다."]
embeddings = embedder.encode(sentences)
print(embeddings)

from sentence_transformers import util

# 두 문장의 유사도 계산
similarity = util.cos_sim(embeddings[0], embeddings[1])
print(f"Similarity: {similarity}")


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# 모델 로드
tokenizer = AutoTokenizer.from_pretrained("jhgan/ko-sroberta-multitask")
model = AutoModel.from_pretrained("jhgan/ko-sroberta-multitask")

# 테스트 문장
text = "한국어 문장을 테스트합니다."
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)

# 출력 텐서 확인
print(outputs)
